In [3]:
import os
import json
import cv2
from matplotlib import pyplot as plt
from dataset import read_wider_meta

from utils import letterbox_pad

### Calculate and write meta

In [2]:
data_root = './data/'

# calculate
# train_meta = read_wider_meta(data_root, phase='train')
# val_meta = read_wider_meta(data_root, phase='val')
# test_meta = read_wider_meta(data_root, phase='test')

# write
# with open('./data/train.json', 'w') as fwrite:
#     json.dump(train_meta, fwrite)
    
# with open('./data/val.json', 'w') as fwrite:
#     json.dump(val_meta, fwrite)

# with open('./data/test.json', 'w') as fwrite:
#     json.dump(test_meta, fwrite)

### Read meta

In [4]:
with open('./data/train.json', 'r') as fread:
    train_meta = json.load(fread)
    
with open('./data/val.json', 'r') as fread:
    val_meta = json.load(fread)
    
with open('./data/test.json', 'r') as fread:
    test_meta = json.load(fread)

### Dataset

In [4]:
from torchvision.transforms import Compose
from dataset import WIDERdataset, ToTensor

%matplotlib inline

train_transforms = Compose([
    ToTensor()
])

train_dataset = WIDERdataset('./data/train.json', 'train', 608, train_transforms)
img, targets = train_dataset.__getitem__(2)

self.transorfms


### Training loop

In [1]:
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torchvision.transforms import Compose

from dataset import WIDERdataset, ToTensor
from darknet import Darknet

cfg_path = './cfg/yolov3_608x608.cfg'
model_width = 608
batch_size = 1
shuffle = True
num_workers = 1
device = torch.device('cpu')

model = Darknet(cfg_path)

train_transforms = Compose([
    ToTensor()
])
train_dataset = WIDERdataset(
    './data/train.json', 'train', model_width, train_transforms
)
train_loader = DataLoader(
    train_dataset, batch_size, shuffle, 
    num_workers=num_workers, collate_fn=train_dataset.collate_wider
)

optimizer = Adam(model.parameters())

model.train()

for i, (images, targets) in enumerate(tqdm(train_loader)):
    optimizer.zero_grad()
    
    images = images.to(device)
    targets = targets.to(device)
    
    _, loss = model(images, targets, device)
    
    loss.backward()
    
    optimizer.step()
    print(loss / batch_size)
    
next(iter(train_loader))

TypeError: __init__() missing 1 required positional argument: 'cfg_path'